In [1]:
import time
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from datetime import date
import pandas as pd
import os

10

In [ ]:
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))

outputpath = 'C:/Users/dimut/Box/Home PC/Data Files/glassdoor/reviews/dec2023'

In [ ]:

driver.get("https://www.glassdoor.com/profile/login_input.htm")

time.sleep(3)
driver.find_element(By.ID, 'inlineUserEmail').send_keys('sumana.gamage23@gmail.com')
driver.find_element(By.ID, 'inlineUserEmail').send_keys(Keys.ENTER)

time.sleep(3)
driver.find_element(By.ID, 'inlineUserPassword').send_keys('cubs123##')
driver.find_element(By.ID, 'inlineUserPassword').send_keys(Keys.ENTER)

time.sleep(3)

urls = pd.read_csv("C:/Users/dimut/Box/Home PC/Data Files/tech_cos_with_glassdoor_link.csv")
urls = urls[urls['glassdoor_url'].str.len() > 1]
file_list = os.listdir('C:/Users/dimut/Box/Home PC/Data Files/glassdoor/reviews/')
file_list = [x.split('.')[0] for x in file_list]
urls = urls.loc[~urls['Symbol'].str.contains('|'.join(file_list))]
# urls = urls.iloc[0:100]
print(len(urls.index))

company_counter = 0
for index, row in urls.iterrows():

    company_counter = company_counter + 1
    print(company_counter)

    # if company_counter % 50 == 0:
    #     cont = input("continue? (y/n) ")
    #
    #     if cont == "n":
    #         break

    ticker = row['Symbol']
    url = row['glassdoor_url']

    print(ticker)
    driver.get(url)
    time.sleep(3)

    try:
        overallReviewCount = int(
            driver.find_element(By.CSS_SELECTOR, '[data-test="overallReviewCount"]').find_elements(By.TAG_NAME,
                                                                                                   'strong')[
                0].text.replace(",", ""))
    except:
        print('overallReviewCount int')
        continue

    if overallReviewCount < 50:
        print('overallReviewCount < 50')
        continue

    f = open(output_path + 'reviews/' + ticker + '.txt', 'a', encoding='utf-8')

    page_nos = range(1, round(overallReviewCount / 10) - 1, max(round(overallReviewCount / 200), 2))

    for pno in page_nos:

        temp_url = url[0:(len(url) - 4)] + "_P" + str(pno+1) + ".htm?sort.sortType=RD"
        print(temp_url)
        try:
            driver.get(temp_url)
        except:
            try:
                time.sleep(5)
                driver.get(temp_url)
            except:
                print(temp_url + " failed")
                continue
        time.sleep(3)

        try:
            gdReview = driver.find_elements(By.CLASS_NAME, "gdReview")

            if len(gdReview) == 0:
                break

            time.sleep(3)

            for gdr in gdReview:
                time.sleep(1)
                try:
                    title = gdr.find_element(By.TAG_NAME, 'h2').text
                    rating = gdr.find_element(By.CLASS_NAME, "ratingNumber").text
                    current_employee = gdr.find_element(By.CLASS_NAME, "pt-xsm.pt-md-0.css-1qxtz39.eg4psks0").text
                    try:
                        date_jobtitle = gdr.find_element(By.CLASS_NAME, "authorInfo").text
                    except:
                        date_jobtitle = gdr.find_element(By.CLASS_NAME,
                                                         "common__EiReviewDetailsStyle__newUiJobLine").text
                    review_body_text = gdr.find_element(By.CLASS_NAME, "reviewBodyCell ").text
                    reviewBody = gdr.find_element(By.CLASS_NAME, "reviewBodyCell ").find_elements(By.CLASS_NAME,
                                                                                                  "SVGInline-svg")

                    if len(reviewBody) == 3:
                        try:
                            review1 = reviewBody[0].find_element(By.TAG_NAME, "path").get_attribute("d")[0:20]
                        except:
                            if len(reviewBody[0].find_elements(By.TAG_NAME, "circle")) == 1:
                                review1 = "circle"
                            elif len(reviewBody[0].find_elements(By.TAG_NAME, "rect")) == 1:
                                review1 = "rect"
                            else:
                                review1 = ""
                        try:
                            review2 = reviewBody[1].find_element(By.TAG_NAME, "path").get_attribute("d")[0:20]
                        except:
                            if len(reviewBody[1].find_elements(By.TAG_NAME, "circle")) == 1:
                                review2 = "circle"
                            elif len(reviewBody[1].find_elements(By.TAG_NAME, "rect")) == 1:
                                review2 = "rect"
                            else:
                                review2 = ""
                        try:
                            review3 = reviewBody[2].find_element(By.TAG_NAME, "path").get_attribute("d")[0:20]
                        except:
                            if len(reviewBody[2].find_elements(By.TAG_NAME, "circle")) == 1:
                                review3 = "circle"
                            elif len(reviewBody[2].find_elements(By.TAG_NAME, "rect")) == 1:
                                review3 = "rect"
                            else:
                                review3 = ""

                        pros = gdr.find_element(By.CSS_SELECTOR, '[data-test="pros"]').text
                        cons = gdr.find_element(By.CSS_SELECTOR, '[data-test="cons"]').text

                        work_life_bal = ""
                        culture_value = ""
                        diversity = ""
                        carrier = ""
                        compensation = ""
                        management = ""

                        if len(gdr.find_elements(By.CLASS_NAME, 'tooltipContainer')) == 1 and len(
                                gdr.find_element(By.CLASS_NAME, 'tooltipContainer').find_elements(By.TAG_NAME,
                                                                                                  "li")) == 6:
                            lis = gdr.find_element(By.CLASS_NAME, 'tooltipContainer').find_elements(By.TAG_NAME, "li")
                            li_count = 1
                            for li in lis:
                                if li_count == 1:
                                    work_life_bal = li.find_elements(By.TAG_NAME, "div")[1].get_attribute('class')
                                if li_count == 2:
                                    culture_value = li.find_elements(By.TAG_NAME, "div")[1].get_attribute('class')
                                if li_count == 3:
                                    diversity = li.find_elements(By.TAG_NAME, "div")[1].get_attribute('class')
                                if li_count == 4:
                                    carrier = li.find_elements(By.TAG_NAME, "div")[1].get_attribute('class')
                                if li_count == 5:
                                    compensation = li.find_elements(By.TAG_NAME, "div")[1].get_attribute('class')
                                if li_count == 6:
                                    management = li.find_elements(By.TAG_NAME, "div")[1].get_attribute('class')
                                # css-1nuumx7 e1hd5jg10 4 stars,s88v13 5 stars, 18v8tui 2, vl2edp 3, xd4dom 1
                                li_count = li_count + 1

                        f.write(
                            str(pno) + "|-|" + ticker + "|-|" + title + "|-|" + rating + "|-|" + current_employee + "|-|"
                            + date_jobtitle + "|-|" + review_body_text.replace("\n",
                                                                               "---") + "|-|" + review1 + "|-|" + review2
                            + "|-|" + review3 + "|-|" + pros + "|-|" + cons + "|-|" + work_life_bal + "|-|" + culture_value
                            + "|-|" + diversity + "|-|" + carrier + "|-|" + compensation + "|-|" + management + "\n")
                    else:
                        print("Error")
                        break
                except Exception as e:
                    time.sleep(10)
                    print("skip")
                    # nextButton = driver.find_elements(By.CLASS_NAME, 'nextButton')
                    # if len(nextButton) == 1:
                    #     if nextButton[0].is_enabled():
                    #         page_no = page_no + 1
                    #         nextButton[0].click()
                    #         time.sleep(5)
                    break


        except:
            break

    f.close()
    time.sleep(30)
driver.close()
